In [ ]:
"""
This notebook is used to play with the dataset, displaying images and understand the structure of the dataset. 
Note: we can use this to check if duplicate images also have duplicate captions.
"""
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import numpy as np

In [ ]:
# specify dataset and paths to work on
DATASET_ENTITY_COUNT = 20_000 # 1_900_000 20_000 222
DATASET_BASE_PATH = f"/Users/yavuz/data/LAION-{DATASET_ENTITY_COUNT}/"

METADATA_PATH = DATASET_BASE_PATH + "metadata.parquet"
IMAGES_PATH = DATASET_BASE_PATH + "images/"

vector_path = DATASET_BASE_PATH + "vectors/"

assert os.path.exists(METADATA_PATH)
assert os.path.exists(IMAGES_PATH)
assert os.path.exists(vector_path)

In [55]:
metadata=pd.read_parquet(METADATA_PATH)
metadata["TEXT"]

0                                                               Anhui Mountains
1                                                    Acute pain in a woman knee
2                                    Essentials Barnwood 70-inch TV Media Stand
3        Ben Affleck Could Be Latest Addition To <em>The Great Gatsby</em> Cast
4                                        Minnesota Department of Transportation
                                          ...                                  
12619                                               Letters over the Wall cover
12620            Http Www Digsdigs Com 33 Sweet Shabby Chic Bedroom Decor Ideas
12621                                             TOTTENHAM 940 BALL CAP - GOLD
12622        Clown toy color vector illustration Royalty Free Stock Photography
12623                                              Rivendale 800XL AirBeam tent
Name: TEXT, Length: 12624, dtype: object

In [ ]:
# display single match
id = 69
metadata.iloc[id]

In [ ]:
id, metadata["TEXT"][id], metadata["URL"][id], metadata["index"][id]

In [ ]:
def get_image_path(vector_id: int, images_path: str):
    """
    Given a vector id and base images path (IMAGES_PATH), returns the image.
    """
    shard = str(vector_id // 10000).zfill(5)
    index = str(vector_id % 10000).zfill(4)
    image_path = f"{images_path}/{shard}/{shard}{index}.jpg"
    return image_path

In [ ]:
from IPython.display import display, Image

def get_image(vector_id: int, images_path: str):
    """
    Given a vector id and base images path (IMAGES_PATH), returns the image.
    """
    shard = str(vector_id // 10000).zfill(5)
    index = str(vector_id % 10000).zfill(4)
    image_path = f"{images_path}/{shard}/{shard}{index}.jpg"
    return Image(filename=image_path) 
display(get_image(id, IMAGES_PATH))

In [ ]:
# load vectors for id
text_vector_path = vector_path+"text_vectors"
image_vector_path = vector_path+"image_vectors"

text_vectors = np.load(text_vector_path+".npy")
image_vectors = np.load(image_vector_path+".npy")

In [ ]:
# dot it with itself
np.dot(text_vectors[id], text_vectors[id])
text_vectors[id]

In [ ]:
np.dot(image_vectors[id], image_vectors[id])
image_vectors[id]

In [ ]:
# look at placeholder images:
placeholder_images_file_name = "placeholder_images-ALLPAIRS-3percent-duplicates-bad.npy"
placeholder_images = np.load(vector_path+placeholder_images_file_name)
placeholder_images.shape

In [ ]:
# play with audio generation
from diffusers import AudioLDM2Pipeline
import torch
import scipy

In [ ]:
repo_id = "cvssp/audioldm2"
pipe = AudioLDM2Pipeline.from_pretrained(repo_id, torch_dtype=torch.float32)

if torch.cuda.is_available():
    pipe = pipe.to("cuda")

In [ ]:
def generate_audio_from_text(text: str, pipe: AudioLDM2Pipeline):
    prompt = "Sound associateed with: " + text
    negative_prompt = "Low quality."

    # run the generation
    audio = pipe(
        prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=10, # original was 200
        audio_length_in_s=3.0, #original was 5.0
        num_waveforms_per_prompt=2, #original was 3
    )
    return audio

In [ ]:
audio = generate_audio_from_text("Oval_drum_small2", pipe)

In [ ]:
a=audio.audios

In [ ]:
scipy.io.wavfile.write("audio.wav", rate=16000, data=a[0])

In [ ]:
pipe.vae

In [ ]:
vae_encoder = pipe.vae.encoder
a[0]

In [ ]:
audio_tensor = a[0]
latent_representation = vae_encoder(audio_tensor)

In [ ]:
latent_representation = vae_encoder(a)

In [ ]:
vae_encoder = pipe.vae

In [ ]:
audio[2]

In [ ]:
audio2 = generate_audio_from_text("Oval_drum_small2", pipe)

In [ ]:
audio2.audios

In [ ]:
actual_audio2= audio2.audios

In [ ]:
# play with video generation
import torch
from diffusers import CogVideoXImageToVideoPipeline
from diffusers.utils import export_to_video, load_image

In [ ]:
i = 0
text = metadata["TEXT"].iloc[i]
image_path = get_image_path(i, IMAGES_PATH)
print(text)
display(get_image(i, IMAGES_PATH))

In [ ]:
prompt = "Video of: " + text
image = load_image(image_path)

In [ ]:
pipe = CogVideoXImageToVideoPipeline.from_pretrained(
    "THUDM/CogVideoX-5b-I2V",
    torch_dtype=torch.bfloat16
)

In [ ]:
# reduce memory requirements 
pipe.vae.enable_tiling()
pipe.vae.enable_slicing()

video = pipe(
    prompt=prompt,
    image=image,
    num_videos_per_prompt=1,
    num_inference_steps=50,
    num_frames=49,
    guidance_scale=6,
    generator=torch.Generator(device="cuda").manual_seed(42),
).frames[0]

In [ ]:
export_to_video(video, "output.mp4", fps=8)